In [3]:
import pandas as pd
import json

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
# Load the Excel file
file_path = './data.xlsx'  # Replace with your actual file path

df_coffee = pd.read_excel(file_path, sheet_name='2-BeverageList', header=2)  # Replace 'Sheet1' with the actual sheet name
df_coffee.columns = df_coffee.columns.str.lower().str.replace(' ', '_').str.replace('#n/d', '')

# Read the Excel file
df_impact = pd.read_excel(file_path, sheet_name='6-AppDatabase', header=2)  # Replace 'Sheet1' with the actual sheet name
df_impact.columns = df_impact.columns.str.lower().str.replace(' ', '-') #Kebab case

df_coffee.head()

,serve-id,recipe-id,retail-name,salepoint-id,coffee-details,unit,market_price,true_cost,true_price
0,ch-epfl-klee_ris-cof,ris-cof,Ristretto,ch-epfl-klee,To fill,CHF/serve,1.6,0.847856,2.447856
1,ch-epfl-klee_ris-decaf,ris-decaf,"Ristretto, décaféiné",ch-epfl-klee,To fill,CHF/serve,1.6,0.766473,2.366473
2,ch-epfl-klee_esp-cof,esp-cof,Espresso,ch-epfl-klee,To fill,CHF/serve,1.6,0.847856,2.447856
3,ch-epfl-klee_esp-decaf,esp-decaf,"Espresso, décaféiné",ch-epfl-klee,To fill,CHF/serve,1.6,0.766473,2.366473
4,ch-epfl-klee_caf-cof,caf-cof,Café,ch-epfl-klee,To fill,CHF/serve,1.6,0.847856,2.447856


In [6]:
def format_data(group):
    # Group by 'stage' and 'impact-category' and collect details under each combination
    grouped_impacts = group.groupby(['stage', 'impact-category']).apply(lambda x: {
        'stage': x.iloc[0]['stage'],  # Add stage as a key
        'impact-category': x.iloc[0]['impact-category'],  # Add impact-category as a key
        'impact-value': round(x['impact-value'].sum(), 4),  # Sum and format impact values to 4 digits
        'cost-value': round(x['cost-value'].sum(), 4),  # Sum and format cost values to 4 digits
        'details': x.apply(lambda row: {
            'indicators': row['indicators'],
            'unit': row['unit'],
            'impact-value': round(row['impact-value'], 4),  # Format impact-value to 4 digits
            'cost-value': round(row['cost-value'], 4),  # Format cost-value to 4 digits
            'impact-definition': row['impact-definition'],
            'reference': row['references']
        }, axis=1).tolist()  # Create a list of impacts under "details"
    }).reset_index(drop=True).tolist()  # Convert to list of dictionaries

    # Return structured data with product-related info and grouped impacts
    return {
        'id': group.iloc[0]['serve-id'],  # Adding ID for each serve product
        'id-sale-point': group.iloc[0]['id-salepoint'],
        'recipe-id': group.iloc[0]['recipe-id'],
        'product-id': group.iloc[0]['product-id'],
        'product-name': group.iloc[0]['product-name'],
        'recipe': float(f"{float(group.iloc[0]['recipe']):.4f}"),  # Format recipe to 2 decimal places
        'impacts': grouped_impacts  # Nested impacts
    }

formatted_data = df_impact.groupby('serve-id').apply(format_data, include_groups=True).tolist()


In [7]:
def parse_coffee_details(row):
    """
    Parses the coffeeDetails field to extract information about decaf, milk, and sugar.
    """
    coffee_details = row['recipe-id'].lower()
    
    # Initialize flags and types
    is_decaf = False
    has_milk = False
    milk_type = None

    # Check for decaffeination
    if 'decaf' in coffee_details or 'deca' in coffee_details: 
        is_decaf = True

    # Check for milk
    milk_types = [
        'cow',
        'clf',
        'oat',
    ]

    for key in milk_types:
        if key in coffee_details:
            has_milk = True
            milk_type = key
            break

    recipe = coffee_details.split('-')[0]
    return {
        'recipe': recipe,
        'is_decaf': is_decaf,
        'has_milk': has_milk,
        'milkType': milk_type
    }


In [8]:
def format_coffee_data(row):
    parsed = parse_coffee_details(row)
    return {
        'serveId': row['serve-id'],
        'recipeId': row['recipe-id'],
        'retailName': row['retail-name'],
        'salePointId': row['salepoint-id'],
        'coffeeDetails': row['coffee-details'],
        'marketPrice': row['market_price'],
        'trueCost': row['true_cost'] if pd.notnull(row['true_cost']) else 0.0,
        'truePrice': row['true_price'] if pd.notnull(row['true_price']) else 0.0,
        'isDecaf': parsed['is_decaf'],
        'hasMilk': parsed['has_milk'],
        'milkType': parsed['milkType'],
        'mainRecipe': parsed['recipe']
    }

In [9]:
coffee_data = pd.DataFrame(df_coffee.apply(format_coffee_data, axis=1).tolist())


In [12]:
output_csv_file = '../public/data/coffe_data.csv'
coffee_data.to_csv(output_csv_file, index=False)

print(coffee_data.head())


                  serveId   recipeId            retailName   salePointId  \
0    ch-epfl-klee_ris-cof    ris-cof             Ristretto  ch-epfl-klee   
1  ch-epfl-klee_ris-decaf  ris-decaf  Ristretto, décaféiné  ch-epfl-klee   
2    ch-epfl-klee_esp-cof    esp-cof              Espresso  ch-epfl-klee   
3  ch-epfl-klee_esp-decaf  esp-decaf   Espresso, décaféiné  ch-epfl-klee   
4    ch-epfl-klee_caf-cof    caf-cof                  Café  ch-epfl-klee   

  coffeeDetails  marketPrice  trueCost  truePrice  isDecaf  hasMilk milkType  \
0       To fill          1.6  0.847856   2.447856    False    False     None   
1       To fill          1.6  0.766473   2.366473     True    False     None   
2       To fill          1.6  0.847856   2.447856    False    False     None   
3       To fill          1.6  0.766473   2.366473     True    False     None   
4       To fill          1.6  0.847856   2.447856    False    False     None   

  mainRecipe  
0        ris  
1        ris  
2        esp  
3 

In [69]:
with open('formatted_coffee_impact_data.json', 'w') as f:
    json.dump(formatted_data, f, indent=4)

# Print the formatted data to inspect
print(json.dumps(formatted_data, indent=4))

[
    {
        "id": "ch-epfl-klee_caf-cof",
        "id-sale-point": "ch-epfl-klee",
        "recipe-id": "caf-cof",
        "product-id": "cof-braz-ara-conv",
        "product-name": "Coffee , Brazil, Arabica, none",
        "recipe": 2.7,
        "impacts": [
            {
                "stage": "Cultivation",
                "impact-category": "Biodiversity",
                "impact-value": 37.3819,
                "cost-value": 0.4981,
                "details": [
                    {
                        "indicators": "eco-costs of land-use (brazil) [m2/kg]",
                        "unit": "m2/kg",
                        "impact-value": 6.15,
                        "cost-value": 0.088,
                        "impact-definition": "To fill",
                        "reference": " Ecoinvent (3.8), ReCiPe 2016 Midpoint (H), Coffee green bean production, arabica (Brazil), Fonctional unit: 1 kg grean coffe bean, Sustainability Impact Metrics (2024): https://www.ecocostsvalue